In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# load data
train_path, test_path = "../input/titanic/train.csv", "../input/titanic/test.csv"
train_df = pd.read_csv(train_path, index_col="PassengerId")
test_df  = pd.read_csv(test_path, index_col="PassengerId")
train_df.info()

# merge train test dataset and assign y
main_df = pd.concat([train_df,test_df])
y = train_df.Survived

In [ ]:
main_df.Age.fillna(main_df.Age.value_counts().head(1).values[0], inplace=True)
main_df.Embarked.fillna(main_df.Embarked.value_counts().head(1).values[0], inplace=True)
main_df.Fare.fillna(main_df.Fare.value_counts().head(1).values[0], inplace=True)
main_df.drop("Cabin",axis=1,inplace=True)
test_df.drop("Cabin",axis=1,inplace=True)

In [ ]:
main_df.isnull().sum()

In [ ]:
main_df["family_size"] = main_df.SibSp + main_df.Parch + 1
main_df["last_name"] = main_df.Name.apply(lambda x: x.split(",")[0])
main_df["title_name"] = main_df.Name.apply(lambda x: x.split(",")[1].split(".")[0])
main_df.drop("Name",axis=1,inplace=True)

In [ ]:
from pandas.api.types import is_string_dtype
for col in main_df.drop("Survived",axis=1).columns:
    if is_string_dtype(main_df[col]):
        main_df[col] = main_df[col].astype("category").cat.codes

In [ ]:
from itertools import combinations as cb
temp_features = main_df.drop("Survived".split(),axis=1)
cb_columns = []
for loop in range(1, len(temp_features)+1):
    cb_columns += cb(temp_features,loop)
print(cb_columns[:5])

In [ ]:
main_df[-5:]

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from datetime import datetime
now = datetime.now()
model_evaluation = {"features":[],"accuracy":[]}

for features in cb_columns:
    model=None
    X=main_df[:len(train_df)][list(features)]
    y=train_df.Survived
    train_X,test_X,train_y,test_y = train_test_split(X,y,test_size=0.2,train_size=0.8,random_state=1)
    
    model = RandomForestRegressor()
    model.fit(train_X,train_y)
    model_evaluation["features"].append(" ".join(features))
    
    df = pd.DataFrame({"a":test_y ,"b":model.predict(test_X).round().astype("int")})
    model_evaluation["accuracy"].append(len(df[df.a==df.b]) / len(df))

models_df = pd.DataFrame(model_evaluation)
models_df
print("time spend:", now-datetime.now())

In [ ]:
models_df.sort_values("accuracy",ascending=False)

In [ ]:
# Pclass Sex Age Ticket family_size
final_features = models_df.sort_values("accuracy",ascending=False).iloc[0,:].features.split()
#final_features = "Pclass Sex Age Ticket family_size".split()
X=main_df[:len(train_df)][final_features]
y=train_df.Survived
train_X,test_X,train_y,test_y = train_test_split(X,y,test_size=0.8,train_size=0.2,random_state=1)
model_evaluation = {"features":[],"accuracy":[]}

model = RandomForestRegressor(n_estimators=100)
model.fit(X,y)
model_evaluation["features"].append(" ".join(final_features))

df = pd.DataFrame({"a":y ,"b":model.predict(X).round().astype("int")})
model_evaluation["accuracy"].append(len(df[df.a==df.b]) / len(df))

models_df = pd.DataFrame(model_evaluation)
models_df

In [ ]:
submit = pd.DataFrame({"PassengerId":main_df[891:].index.astype("int"), "Survived":model.predict(main_df[891:][final_features]).round().astype("int")})
submit.to_csv("submissionJavier_Vallejos.csv",index = False)

In [ ]:
test_df.index
main_df[891:]

In [ ]:
now-datetime.now()

In [ ]:
pd.read_csv("./submissionJavier_Vallejos.csv")